# Fundamental Analysis

In [41]:
import pandas as pd
from vnstock import Vnstock
# Set date range
start_date = '2024-01-01'
end_date = '2025-03-19'
interval = '1D'
symbol='REE'
stock = Vnstock().stock(symbol=symbol, source='VCI')
candle_df = stock.quote.history(start= start_date, end= end_date)

In [42]:
CashFlow = stock.finance.cash_flow(period='year')
#CashFlow.to_csv('./outputs/CashFlow.csv')

## Transpose the CF dataframe 

In [43]:
#CashFlow
CashFlow_transposed = CashFlow.T
CashFlow_transposed.columns = CashFlow['yearReport']
# Drop the duplicate 'yearReport' row
CashFlow_transposed = CashFlow_transposed.drop('yearReport')


In [44]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Normalize values to billions
data = CashFlow['Net cash inflows/outflows from operating activities'] / 1000_000_000

# Sort data by year to ensure proper ordering
sorted_indices = CashFlow['yearReport'].argsort()
years = CashFlow['yearReport'].iloc[sorted_indices].tolist()
sorted_data = data.iloc[sorted_indices].tolist()

# Create the figure
fig = go.Figure()

# Add the bar chart
fig.add_trace(go.Bar(
    x=years,
    y=sorted_data,
    text=[f'{v:.2f}' for v in sorted_data],
    textposition='outside',
    marker_color='rgb(66, 135, 245)',
    marker_line_color='lightgrey',
    marker_line_width=1.5,
    opacity=0.8
))

# Update layout
fig.update_layout(
    title='Net Operating Cashflow',
    xaxis_title='Years',
    yaxis_title='Value (Billions)',
    plot_bgcolor='white',
    margin=dict(l=50, r=50, t=80, b=50),
    height=500,
    width=800
)

# Clean up the axes
fig.update_xaxes(
    tickangle=45,
    showgrid=False
)

fig.update_yaxes(
    showgrid=False,
    gridcolor='lightgray',
    gridwidth=0.5
)

# Show the plot
fig.show()

In [45]:
Ratio = stock.finance.ratio(period='year', lang='vi', dropna=True)

2025-05-17 19:11:25 - vnstock.explorer.vci.financial - ERROR - Error processing financial report data: Failed to fetch data: 405 - Not Allowed
2025-05-17 19:11:25 - vnstock.explorer.vci.financial - ERROR - Error retrieving financial ratios: Failed to fetch data: 405 - Not Allowed
2025-05-17 19:11:25 - vnstock.common.data.data_explorer - ERROR - Lỗi khi truy xuất dữ liệu ratio: Failed to fetch data: 405 - Not Allowed


ConnectionError: Failed to fetch data: 405 - Not Allowed

### Transpose the data frame to display on the web.

In [ ]:
Ratio_transposed = Ratio.T
Ratio_transposed.columns=Ratio_transposed.iloc[1]
Ratio_transposed = Ratio_transposed.iloc[3:]

In [ ]:
# Example: Select the 'ROE (%)' column under 'Chỉ tiêu khả năng sinh lợi'
dividend_yield = Ratio[('Chỉ tiêu khả năng sinh lợi', 'Tỷ suất cổ tức (%)')]
Outstanding_Shares = Ratio[('Chỉ tiêu định giá', 'Số CP lưu hành (Triệu CP)')]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define the five most meaningful metrics (six columns for all pairwise relationships)
selected_cols = [
    ('Chỉ tiêu khả năng sinh lợi', 'ROE (%)'),
    ('Chỉ tiêu cơ cấu nguồn vốn', 'Nợ/VCSH'),
    ('Chỉ tiêu hiệu quả hoạt động', 'Vòng quay tài sản'),
    ('Chỉ tiêu khả năng sinh lợi', 'Biên lợi nhuận ròng (%)'),
    ('Chỉ tiêu thanh khoản', 'Chỉ số thanh toán hiện thời'),
    ('Chỉ tiêu định giá', 'P/S'),
]

# Subset the DataFrame
df_pair = Ratio[selected_cols].copy()

# Use only the second part of each column tuple (the metric name)
df_pair.columns = [col[1] for col in df_pair.columns]

# Optional: Remove rows with missing values for these columns
#df_pair = df_pair.dropna()

# Create the pairplot
sns.pairplot(df_pair, diag_kind='kde', corner=True)
plt.suptitle('Pairplot of Key Financial Metrics', y=1.02)
plt.tight_layout()
plt.show()

### Transform the balance sheet from long format to wide format. 

In [ ]:
def BS_wide(stock=None):
    """
    Transform balance sheet data into a wide format with years as columns and metrics as rows.
    
    Parameters:
    -----------
    stock : object, default=None
        The stock ticker object containing financial data.
        If None, will create a default stock object for REE from VCI source.
    
    Returns:
    --------
    pandas.DataFrame
        Transformed balance sheet with years as columns and financial metrics as rows
    """
    
    # Create default stock object if not provided
    if stock is None:
        stock = Vnstock().stock(symbol= symbol, source='VCI')
    
    # Get the balance sheet data
    BS = stock.finance.balance_sheet(period='year', lang='en', dropna=True)
    
    # Transpose the DataFrame
    BS_wide = BS.T
    
    # Promote header by setting column names using the second row (index 1)
    BS_wide.columns = BS_wide.iloc[1]
    
    # Keep only the data rows (skip the first 3 rows)
    BS_wide = BS_wide.iloc[3:]
    
    return BS_wide

In [ ]:
# BS_wide(stock)

In [ ]:
BalanceSheet = stock.finance.balance_sheet(period='year', lang='en', dropna=True)
BalanceSheet_Transposed = BalanceSheet.T
BalanceSheet_Transposed.columns = BalanceSheet_Transposed.iloc[1]
BalanceSheet_Transposed = BalanceSheet_Transposed.iloc[3:]


### Transform Income statement from long format to wide format. 

In [ ]:
IncomeStatement = stock.finance.income_statement(period='year', lang='en', dropna=True)
IncomeStatement_Transpose= IncomeStatement.T
IncomeStatement_Transpose.columns = IncomeStatement_Transpose.iloc[1]
IncomeStatement_Transpose = IncomeStatement_Transpose.iloc[3:]


In [ ]:
from vnstock import Vnstock
import warnings
warnings.filterwarnings("ignore")
company = Vnstock().stock(symbol=symbol, source='TCBS').company


In [ ]:
# Levered Free Cash Flow (accounts for debt repayments/receipts)
CashFlow['Levered Free Cash Flow'] = (
    CashFlow['Net cash inflows/outflows from operating activities'] 
    - CashFlow['Purchase of fixed assets']
    + CashFlow['Proceeds from disposal of fixed assets']
    - (CashFlow['Repayment of borrowings'] - CashFlow['Proceeds from borrowings'])
)

In [ ]:
dividend_coverage_ratio = CashFlow['Levered Free Cash Flow'] / CashFlow['Dividends paid'].abs()

# Using Market value cost of debt

In [ ]:
# Step 1: Get book values from Balance Sheet
short_term_debt = BalanceSheet['Short-term borrowings (Bn. VND)']
long_term_debt = BalanceSheet['Long-term borrowings (Bn. VND)']
total_debt = short_term_debt + long_term_debt
book_equity = BalanceSheet["OWNER'S EQUITY(Bn.VND)"]

# Step 2: Get market values
# Market capitalization for equity
market_value_of_equity = Ratio[('Chỉ tiêu định giá', 'Vốn hóa (Tỷ đồng)')]  # Market capitalization

# Use book value of debt as a proxy for market value of debt
# (In practice, we'd prefer bond prices or yield-based valuation if available)
market_value_of_debt = total_debt

# Calculate total market capital and weights
total_market_capital = market_value_of_equity + market_value_of_debt
market_weight_of_debt = market_value_of_debt.div(total_market_capital).fillna(0)
market_weight_of_equity = market_value_of_equity.div(total_market_capital).fillna(0)

# Step 3: Market-based cost of debt
# Option 1: If you have specific bond yield data (example values)
# In reality, this would vary by company or be derived from external data sources
base_interest_rate = 0.04  # e.g., Vietnamese government bond rate
credit_spread = 0.03       # Credit spread based on company rating
company_bond_yield = base_interest_rate + credit_spread  # = 0.05 (5%)

# Option 2: Use credit rating to determine yield (if available)
# This would be a mapping from credit ratings to yields
# rating_to_yield = {'AAA': 0.035, 'AA': 0.04, 'A': 0.045, 'BBB': 0.05, 'BB': 0.06, 'B': 0.07}
# company_bond_yield = rating_to_yield.get(company_rating, 0.05)  # Default to 5% if rating unknown

# Use fixed rate for simplicity (you would replace this with company-specific data)
market_cost_of_debt = 0.07  # 7% bond yield

# Apply tax shield
statutory_tax_rate = 0.12  # Vietnamese corporate tax rate
after_tax_market_cost_of_debt = market_cost_of_debt * (1 - statutory_tax_rate)

# Step 4: Cost of Equity using CAPM
risk_free_rate = 0.03  # Vietnamese government bond yield

# Option 1: If you have beta data from external sources
# estimated_beta = external_beta_data  # This would be company-specific

# Option 2: Estimate beta using financial leverage
financial_leverage = Ratio[('Chỉ tiêu thanh khoản', 'Đòn bẩy tài chính')]
leverage_mean = financial_leverage.mean()
estimated_beta = 1.0 * financial_leverage.div(leverage_mean).fillna(1.0)  # Default to 1.0 if issues

# Market risk premium
market_risk_premium = 0.05  # Estimated risk premium for Vietnamese market

# Calculate cost of equity using CAPM
cost_of_equity = risk_free_rate + (estimated_beta * market_risk_premium)

# Step 5: Calculate market-based WACC
wacc_market_based = (market_weight_of_debt * after_tax_market_cost_of_debt) + (market_weight_of_equity * cost_of_equity)

# Create a DataFrame with the results
result_df = pd.DataFrame({
    'ticker': BalanceSheet['ticker'],
    'yearReport': BalanceSheet['yearReport'],
    'market_cap': market_value_of_equity,
    'market_debt': market_value_of_debt,
    'market_weight_of_debt': market_weight_of_debt,
    'market_weight_of_equity': market_weight_of_equity,
    'market_cost_of_debt': after_tax_market_cost_of_debt,
    'beta': estimated_beta,
    'cost_of_equity': cost_of_equity,
    'wacc_market_based': wacc_market_based
})

# Display first few rows to check the results
print("Market-Based WACC Calculation Results:")
print(result_df[['ticker', 'yearReport', 'wacc_market_based']].head())


# Calculate average
avg_wacc_by_year = result_df['wacc_market_based'].mean()
print("\nAverage Market-Based WACC by Year:")
print(avg_wacc_by_year)


In [ ]:

# Optional: Visualize the distribution of market-based WACC
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(result_df['wacc_market_based'].dropna(), kde=True)
plt.title('Distribution of Market-Based WACC')
plt.xlabel('WACC')
plt.ylabel('Frequency')
plt.axvline(result_df['wacc_market_based'].mean(), color='red', linestyle='--', 
            label=f'Mean WACC: {result_df["wacc_market_based"].mean():.2%}')
plt.legend()
plt.show()



In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy import stats

# Calculate statistics for annotations
wacc_data = result_df['wacc_market_based'].dropna()
wacc_mean = wacc_data.mean()
wacc_median = wacc_data.median()

# Create a figure
fig = go.Figure()

# Add the histogram with custom styling
fig.add_trace(go.Histogram(
    x=wacc_data,
    nbinsx=15,
    name='WACC Distribution',
    histnorm='probability density',
    marker=dict(
        color='rgba(66, 135, 245, 0.7)',
        line=dict(color='rgba(33, 69, 123, 1)', width=1)
    ),
    opacity=0.8
))

# Generate KDE curve manually
kde = stats.gaussian_kde(wacc_data)
x_range = np.linspace(wacc_data.min(), wacc_data.max(), 100)
y_kde = kde(x_range)

# Add the KDE curve
fig.add_trace(go.Scatter(
    x=x_range,
    y=y_kde,
    mode='lines',
    name='Density',
    line=dict(color='rgba(236, 72, 72, 1)', width=3),
))

# Add vertical line for mean
fig.add_vline(
    x=wacc_mean,
    line_width=2,
    line_dash="dash",
    line_color="rgba(236, 72, 72, 1)",
    annotation_text=f"Mean: {wacc_mean:.2%}",
    annotation_position="top right",
    annotation_font_color="rgba(236, 72, 72, 1)",
    annotation_font_size=14
)

# Add vertical line for median
fig.add_vline(
    x=wacc_median,
    line_width=2,
    line_dash="dot",
    line_color="rgba(46, 167, 76, 1)",
    annotation_text=f"Median: {wacc_median:.2%}",
    annotation_position="top left",
    annotation_font_color="rgba(46, 167, 76, 1)",
    annotation_font_size=14
)

# Update layout with modern styling
fig.update_layout(
    title={
        'text': '<b>Distribution of Market-Based WACC</b>',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='#303030')
    },
    xaxis_title={
        'text': '<b>WACC</b>',
        'font': dict(size=16)
    },
    yaxis_title={
        'text': '<b>Density</b>',
        'font': dict(size=16)
    },
    template='plotly_white',
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Arial"
    ),
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='rgba(0, 0, 0, 0.1)',
        borderwidth=1
    ),
    margin=dict(l=80, r=80, t=100, b=80),
    plot_bgcolor='rgba(248, 249, 250, 1)',
    paper_bgcolor='rgba(248, 249, 250, 1)',
    xaxis=dict(
        tickformat='.1%',
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(211, 211, 211, 0.4)',
        zeroline=False
    ),
    yaxis=dict(
        showgrid=True,
        gridwidth=1,
        gridcolor='rgba(211, 211, 211, 0.4)',
        zeroline=False
    ),
    width=900,
    height=600
)

# Add annotations for statistical insights
fig.add_annotation(
    x=0.02,
    y=-0.17,
    xref="paper",
    yref="paper",
    text=f"<b>WACC Statistics:</b> Mean: {wacc_mean:.2%} | Median: {wacc_median:.2%} | Min: {wacc_data.min():.2%} | Max: {wacc_data.max():.2%} | StdDev: {wacc_data.std():.2%}",
    showarrow=False,
    font=dict(size=12, color="#505050"),
    align="left",
    bordercolor="#d3d3d3",
    borderwidth=1,
    borderpad=8,
    bgcolor="rgba(250, 250, 250, 0.9)",
    opacity=0.9
)

# Show the figure
fig.show()

In [ ]:
import pandas as pd
import numpy as np

def dcf_time_series(df, cash_flow_col, date_col, discount_rate, base_date=None):
    """
    Calculate DCF using a pandas DataFrame with dates.
    
    Parameters:
    df (DataFrame): DataFrame containing cash flows and dates
    cash_flow_col (str): Column name for cash flows
    date_col (str): Column name for dates
    discount_rate (float): Annual discount rate
    base_date (datetime, optional): Base date for discounting, defaults to min date
    
    Returns:
    float: Present value of cash flows
    """
    # Ensure dates are datetime objects
    df[date_col] = pd.to_datetime(df[date_col])
    
    if base_date is None:
        base_date = df[date_col].min()
    
    # Calculate years from base date
    df['years'] = (df[date_col] - base_date).dt.days / 365.25
    
    # Calculate discount factors
    df['discount_factor'] = 1 / (1 + discount_rate) ** df['years']
    
    # Calculate present values
    df['present_value'] = df[cash_flow_col] * df['discount_factor']
    
    return df['present_value'].sum()

In [ ]:
import pandas as pd

def full_dcf_model(projected_fcf, terminal_growth_rate, wacc, terminal_year):
    """
    Simple DCF model to calculate enterprise value.

    :param projected_fcf: List of projected free cash flows for each forecast year.
    :param terminal_growth_rate: Growth rate applied after the forecast period.
    :param wacc: Weighted Average Cost of Capital used as the discount rate.
    :param terminal_year: Number of years in the forecast period.
    :return: Calculated enterprise value.
    """
    # Calculate the PV of projected cash flows
    present_value_fcf = sum(fcf / ((1 + wacc) ** (i + 1)) for i, fcf in enumerate(projected_fcf))
    
    # Calculate the terminal value
    last_year_fcf = projected_fcf[-1]
    terminal_value = last_year_fcf * (1 + terminal_growth_rate) / (wacc - terminal_growth_rate)
    
    # Present value of the terminal value
    present_value_terminal = terminal_value / ((1 + wacc) ** terminal_year)
    
    # Total enterprise value is the sum of the PV of FCF and PV of terminal value
    enterprise_value = present_value_fcf + present_value_terminal
    
    return enterprise_value

# Ensure CashFlow is a properly defined DataFrame with 'Levered Free Cash Flow'
# Example setup, ensure that 'CashFlow' exists with your data
# CashFlow = pd.DataFrame({'Levered Free Cash Flow': [...]})

# Extract the free cash flow for the stock
free_cash_flow = CashFlow['Levered Free Cash Flow']

# Determine whether to use the mean or a specific value of FCF
# Calculate the mean directly if there are multiple entries for the stock
ticker_fcf = free_cash_flow.mean() if len(free_cash_flow) > 1 else free_cash_flow.iloc[0]

# Define your parameters
forecast_years = 5
terminal_growth_rate = 0.03  # Long-term growth rate
growth_rates = [0.05, 0.06, 0.07, 0.05, 0.04]  # Growth rates for next 5 years

# Retrieve WACC for this single stock from a relevant DataFrame
# Ensure result_df is properly defined
# result_df = pd.DataFrame({'wacc_market_based': [...]})
ticker_wacc = result_df['wacc_market_based'].mean()  # Ensure result_df is defined and contains 'wacc_market_based'

# Project future cash flows
projected_fcf = []
for i, growth in enumerate(growth_rates):
    if i == 0:
        projected_fcf.append(ticker_fcf * (1 + growth))
    else:
        projected_fcf.append(projected_fcf[i-1] * (1 + growth))

# Calculate enterprise value for the single stock
ev = full_dcf_model(projected_fcf, terminal_growth_rate, ticker_wacc, terminal_year=forecast_years)

# Store the results
enterprise_values = {
    'ticker': symbol,  # Replace 'YOUR_TICKER' with the actual stock ticker
    'wacc': ticker_wacc,
    'last_fcf': ticker_fcf,
    'enterprise_value': ev
}

# Convert to DataFrame if desired for structured display
enterprise_values_df = pd.DataFrame([enterprise_values])

# Display the result
print(enterprise_values_df)

In [ ]:
#intrinsic_value = ev - net_debt / Outstanding_Shares
intrinsic_value = ev / Outstanding_Shares
print(intrinsic_value)

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

def create_cashflow_waterfall(df, ticker, year):
    # Filter dataframe for specific ticker and year
    data = df[(df['ticker'] == ticker) & (df['yearReport'] == year)].iloc[0]
    
    # Select components for the waterfall chart
    measures = [
        'Operating profit before changes in working capital',
        'Increase/Decrease in receivables',
        'Increase/Decrease in inventories',
        'Increase/Decrease in payables',
        'Increase/Decrease in prepaid expenses',
        'Interest paid',
        'Business Income Tax paid',
        'Other receipts from operating activities',
        'Other payments on operating activities',
        'Purchase of fixed assets',
        'Proceeds from disposal of fixed assets',
        'Investment in other entities',
        'Proceeds from divestment in other entities',
        'Gain on Dividend',
        'Increase in charter captial',
        'Payments for share repurchases',
        'Proceeds from borrowings',
        'Repayment of borrowings',
        'Dividends paid'
    ]
    
    # Extract values for these measures
    values = [data[measure] for measure in measures]
    
    # Create labels for the chart (shortened for better display)
    labels = [m.split('/')[-1].replace(' activities', '') if '/' in m else m for m in measures]
    labels = [label[:20] + '...' if len(label) > 20 else label for label in labels]
    
    # Set up measure types (relative vs total)
    measure_types = ['relative'] * len(measures)
    
    # Add initial and final cash positions
    labels = ['Initial Balance'] + labels + ['Ending Balance']
    values = [data['Cash and cash equivalents']] + values + [data['Cash and Cash Equivalents at the end of period']]
    measure_types = ['absolute'] + measure_types + ['total']
    
    # Create the waterfall chart with comma-separated numbers
    fig = go.Figure(go.Waterfall(
        name = f"Cashflow Waterfall for {ticker} ({year})",
        orientation = "v",
        measure = measure_types,
        x = labels,
        textposition = "outside",
        # Format numbers with commas as thousands separators
        text = [f"{x:,.1f}" for x in values],
        y = values,
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ))
    
    # Update layout
    fig.update_layout(
        title = f"Cashflow Waterfall Chart for {ticker} ({year})",
        showlegend = False,
        height = 600,
        width = 1000,
        xaxis = dict(
            title = "Cash inflows/outflows",
            tickangle = 45
        ),
        yaxis = dict(
            title = "Amount",
            # Add comma separator to y-axis labels as well
            tickformat = ",.0f"
        )
    )
    
    return fig

# Example usage
# Replace 'REE' and 2024 with your desired ticker and year
cf_year = 2024

fig = create_cashflow_waterfall(CashFlow, 'REE', cf_year)
fig.show()

# If you prefer to save the figure
# fig.write_html("cashflow_waterfall.html")

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def calculate_roce_and_include_roe(BalanceSheet, IncomeStatement, Ratio):
    """
    Calculate ROCE and include ROE from Ratio dataframe with MultiIndex columns.
    
    Parameters:
    -----------
    BalanceSheet : pandas DataFrame
        Balance Sheet data with columns: ticker, yearReport, 'TOTAL ASSETS (Bn. VND)', 'Current liabilities (Bn. VND)'
    IncomeStatement : pandas DataFrame
        Income Statement data with columns: ticker, yearReport, 'Operating Profit/Loss'
    Ratio : pandas DataFrame
        Ratio data with MultiIndex columns including ('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')
        
    Returns:
    --------
    pandas DataFrame
        DataFrame with ROCE and ROE calculations
    """
    # Create a copy to avoid modifying the original dataframe
    BalanceSheet_copy = BalanceSheet.copy()
    
    # Step 1: Calculate Capital Employed
    BalanceSheet_copy['Capital Employed (Bn. VND)'] = (
        BalanceSheet_copy['Long-term borrowings (Bn. VND)'] + 
        BalanceSheet_copy['Short-term borrowings (Bn. VND)'] + 
        BalanceSheet_copy["OWNER'S EQUITY(Bn.VND)"]
    )
    
    # Step 2: Merge Balance Sheet and Income Statement
    merged_df = pd.merge(
        BalanceSheet_copy[['ticker', 'yearReport', 'Capital Employed (Bn. VND)']],
        IncomeStatement[['ticker', 'yearReport', 'Operating Profit/Loss']],
        on=['ticker', 'yearReport'],
        how='inner'
    )
    
    # Step 3: Calculate ROCE
    merged_df['ROCE'] = merged_df['Operating Profit/Loss'] / merged_df['Capital Employed (Bn. VND)']
    
    # Select columns for ROCE calculation
    ROCE_df = merged_df[['ticker', 'yearReport', 'Operating Profit/Loss', 
                         'Capital Employed (Bn. VND)', 'ROCE']]
    ROCE_df = ROCE_df.rename(columns={'Operating Profit/Loss': 'EBIT (Bn. VND)'})
    
    # Step 4: Create a simplified version of Ratio DataFrame for merging
    # Extract the ticker, year, and ROE columns
    ratio_simple = pd.DataFrame({
        'ticker': Ratio[('Meta', 'CP')],
        'yearReport': Ratio[('Meta', 'Năm')],
        'ROE': Ratio[('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')]
    })
    
    # Step 5: Merge with simplified Ratio dataframe to include ROE
    ROCE_df = pd.merge(
        ROCE_df,
        ratio_simple[['ticker', 'yearReport', 'ROE']],
        on=['ticker', 'yearReport'],
        how='left'
    )
    
    return ROCE_df


In [ ]:
def visualize_roce_vs_roe(BalanceSheet, IncomeStatement, Ratio):
    """
    Calculate and visualize the ROCE vs ROE comparison using Plotly.
    
    Parameters:
    -----------
    BalanceSheet : pandas DataFrame
        Balance Sheet data
    IncomeStatement : pandas DataFrame
        Income Statement data
    Ratio : pandas DataFrame
        Ratio data containing ROE with MultiIndex columns
        
    Returns:
    --------
    pandas DataFrame
        DataFrame containing the calculated ROCE and ROE values
    """
    # Calculate ROCE and include ROE
    ROCE_df = calculate_roce_and_include_roe(BalanceSheet, IncomeStatement, Ratio)
    
    # Sort by ticker and year
    ROCE_df = ROCE_df.sort_values(['ticker', 'yearReport'])
    
    # Create a comparison chart for each ticker
    for ticker in ROCE_df['ticker'].unique():
        ticker_data = ROCE_df[ROCE_df['ticker'] == ticker]
        
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        # Add ROCE trace
        fig.add_trace(
            go.Scatter(
                x=ticker_data['yearReport'],
                y=ticker_data['ROCE'],
                name='ROCE',
                mode='lines+markers+text',
                text=[f"{x:.2f}" for x in ticker_data['ROCE']], 
                textposition="top center",
                line=dict(color='rgba(0, 117, 210, 0.9)', width=3),
                marker=dict(
                    size=12, 
                    color='rgba(0, 117, 210, 0.9)',
                    line=dict(color='white', width=2)
                ),
                textfont=dict(color='rgba(0, 117, 210, 1)', size=11, family="Arial Black")
            ),
            secondary_y=False,
        )
        
        # Add shaded area under ROCE for emphasis
        fig.add_trace(
            go.Scatter(
                x=ticker_data['yearReport'],
                y=ticker_data['ROCE'],
                name='ROCE Area',
                mode='none',
                fill='tozeroy',
                fillcolor='rgba(0, 117, 210, 0.1)',
                showlegend=False,
                hoverinfo='skip'
            ),
            secondary_y=False,
        )
        
        # Add ROE trace
        fig.add_trace(
            go.Scatter(
                x=ticker_data['yearReport'],
                y=ticker_data['ROE'],
                name='ROE (%)',
                mode='lines+markers+text',
                text=[f"{x:.1f}%" for x in ticker_data['ROE']],
                textposition="bottom center",
                line=dict(color='rgba(220, 20, 60, 0.9)', width=3),
                marker=dict(
                    size=12, 
                    symbol='diamond',
                    color='rgba(220, 20, 60, 0.9)',
                    line=dict(color='white', width=2)
                ),
                textfont=dict(color='rgba(220, 20, 60, 1)', size=11, family="Arial Black")
            ),
            secondary_y=True,
        )
        
        # Add shaded area under ROE for emphasis
        fig.add_trace(
            go.Scatter(
                x=ticker_data['yearReport'],
                y=ticker_data['ROE'],
                name='ROE Area',
                mode='none',
                fill='tozeroy',
                fillcolor='rgba(220, 20, 60, 0.1)',
                showlegend=False,
                hoverinfo='skip'
            ),
            secondary_y=True,
        )
        
        # Add crossover points as a special marker
        for i in range(1, len(ticker_data)):
            if ((ticker_data['ROCE'].iloc[i-1] > ticker_data['ROE'].iloc[i-1] and 
                 ticker_data['ROCE'].iloc[i] < ticker_data['ROE'].iloc[i]) or
                (ticker_data['ROCE'].iloc[i-1] < ticker_data['ROE'].iloc[i-1] and 
                 ticker_data['ROCE'].iloc[i] > ticker_data['ROE'].iloc[i])):
                
                # This is a crossover point - highlight it
                fig.add_annotation(
                    x=ticker_data['yearReport'].iloc[i],
                    y=ticker_data['ROCE'].iloc[i],
                    text="Crossover Point",
                    showarrow=True,
                    arrowhead=2,
                    arrowsize=1,
                    arrowwidth=2,
                    arrowcolor="#636363",
                    ax=0,
                    ay=-40,
                    bordercolor="#c7c7c7",
                    borderwidth=2,
                    borderpad=4,
                    bgcolor="white",
                    opacity=0.8
                )
        
        # Add figure title with ticker emphasis
        fig.update_layout(
            title={
                'text': f'<b>ROCE vs ROE Comparison</b><br><span style="font-size:16px;">{ticker}</span>',
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'
            },
            template='plotly_white',
            hovermode='x unified',
            margin=dict(t=100, b=120, l=80, r=80),
            hoverlabel=dict(
                bgcolor="white",
                font_size=12,
                font_family="Arial"
            ),
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1,
                bgcolor='rgba(255, 255, 255, 0.7)',
                bordercolor='rgba(0, 0, 0, 0.1)',
                borderwidth=1
            ),
            annotations=[
                dict(
                    x=0.5,
                    y=-0.18,
                    xref='paper',
                    yref='paper',
                    text="<b>Financial Leverage Interpretation</b><br>ROE > ROCE: Effective use of financial leverage | ROCE > ROE: Potential underutilization of debt",
                    showarrow=False,
                    font=dict(size=11, color="#505050"),
                    align="center",
                    bordercolor="#d3d3d3",
                    borderwidth=1,
                    borderpad=8,
                    bgcolor="rgba(250, 250, 250, 0.9)",
                    opacity=0.9
                )
            ]
        )
        
        # Set x-axis title and style
        fig.update_xaxes(
            title_text="<b>Year</b>",
            title_font=dict(size=14),
            showgrid=True,
            gridwidth=1,
            gridcolor='rgba(211, 211, 211, 0.4)',
            showline=True,
            linewidth=2,
            linecolor='rgba(0, 0, 0, 0.3)',
            tickangle=45,
            tickfont=dict(size=12)
        )
        
        # Set y-axes titles and styles
        fig.update_yaxes(
            title_text="<b>ROCE</b>",
            title_font=dict(size=14, color='rgba(0, 117, 210, 1)'),
            showgrid=True,
            gridwidth=1, 
            gridcolor='rgba(211, 211, 211, 0.4)',
            showline=True,
            linewidth=2, 
            linecolor='rgba(0, 117, 210, 0.7)',
            tickformat='.3f',
            tickfont=dict(size=12),
            secondary_y=False
        )
        
        fig.update_yaxes(
            title_text="<b>ROE (%)</b>",
            title_font=dict(size=14, color='rgba(220, 20, 60, 1)'),
            showgrid=False,
            showline=True, 
            linewidth=2, 
            linecolor='rgba(220, 20, 60, 0.7)',
            tickformat='.1f',
            ticksuffix='%',
            tickfont=dict(size=12),
            secondary_y=True
        )
        
        # Add background color gradients
        fig.update_layout(
            plot_bgcolor='rgba(248, 249, 250, 1)',
            paper_bgcolor='rgba(248, 249, 250, 1)',
            width=1000,
            height=650
        )
        
        # Add a watermark or logo in the corner (optional)
        fig.add_annotation(
            x=1.0,
            y=-0.12,
            xref='paper',
            yref='paper',
            text="Financial Analysis",
            showarrow=False,
            font=dict(size=10, color="rgba(150, 150, 150, 0.5)"),
            align="right",
        )
        
        fig.show()
    
    return ROCE_df

In [ ]:
# Calculate ROCE and ROE
ROCE_df = calculate_roce_and_include_roe(BalanceSheet, IncomeStatement, Ratio)
print(ROCE_df.head())

# Visualize ROCE vs ROE comparison
visualize_roce_vs_roe(BalanceSheet, IncomeStatement, Ratio)

# Corporate Finance

In [ ]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd

# Get the years from both DataFrames
ratio_years = Ratio[('Meta', 'Năm')].astype(int).values
result_years = result_df['yearReport'].astype(int).values

# Find common years
common_years = sorted(list(set(ratio_years).intersection(set(result_years))))

# Filter and sort both DataFrames by common years
ratio_filtered = Ratio[Ratio[('Meta', 'Năm')].astype(int).isin(common_years)]
ratio_filtered = ratio_filtered.sort_values(('Meta', 'Năm'))

result_filtered = result_df[result_df['yearReport'].astype(int).isin(common_years)]
result_filtered = result_filtered.sort_values('yearReport')

# Calculate values using the filtered and aligned data
wacc_mean = result_filtered['wacc_market_based'].mean() * 100
roic_values = ratio_filtered[('Chỉ tiêu khả năng sinh lợi', 'ROIC (%)')] * 100
wacc_values = result_filtered['wacc_market_based'] * 100

# Create the figure
fig = go.Figure()

# Add ROIC line
fig.add_trace(go.Scatter(
    x=ratio_filtered[('Meta', 'Năm')],
    y=roic_values,
    name='ROIC (%)',
    line=dict(color='blue')
))

# Add WACC line
fig.add_trace(go.Scatter(
    x=result_filtered['yearReport'],
    y=wacc_values,
    name='WACC (%)',
    line=dict(color='green')
))

# Add horizontal line at WACC mean
fig.add_hline(
    y=wacc_mean,
    line_dash="dash",
    line_color="red",
    annotation_text=f"WACC Mean: {wacc_mean:.2f}%",
    annotation_position="bottom right"
)

# Only add shaded area if we have data points
if len(roic_values) > 0:
    # Create x values for the shaded area
    x_combined = np.concatenate([ratio_filtered[('Meta', 'Năm')], ratio_filtered[('Meta', 'Năm')][::-1]])
    y_upper = np.where(roic_values > wacc_mean, roic_values, wacc_mean)
    y_lower = np.full_like(roic_values, wacc_mean)
    y_combined = np.concatenate([y_upper, y_lower[::-1]])

    # Add the shaded area
    fig.add_trace(go.Scatter(
        x=x_combined,
        y=y_combined,
        fill='toself',
        fillcolor='rgba(0, 100, 255, 0.2)',
        line=dict(width=0),
        showlegend=True,
        name='Economic Value',
        legendgroup='economic_value',
        hoverinfo='skip'
    ))

# Add Economic Value annotation in upper left corner
fig.add_annotation(
    x=0.02,  # Left side
    y=0.95,  # Top side
    xref="paper",  # Use paper coordinates (0-1)
    yref="paper",  # Use paper coordinates (0-1)
    text="<b>Economic Value</b>",
    showarrow=False,
    font=dict(
        size=12,
        color="blue"
    ),
    bgcolor='rgba(0, 100, 255, 0.2)',  # Same as fill color
    bordercolor="blue",
    borderwidth=1,
    borderpad=4,
    opacity=0.8,
    xanchor="left",  # Left-align the text
    yanchor="top"    # Anchor at the top
)

# Update layout with legend at bottom
fig.update_layout(
    title='ROIC (%) vs WACC (%) with Economic Value Zone',
    xaxis_title='Year',
    yaxis_title='Percentage (%)',
    showlegend=True,
    template='plotly_white',
    hovermode='x unified',
    margin=dict(t=50, l=50, r=50, b=80),  # Added more bottom margin for legend
    legend=dict(
        orientation="h",  # Horizontal legend
        yanchor="bottom",
        y=-0.3,  # Position below the x-axis
        xanchor="center",
        x=0.5,   # Center the legend
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    )
)

# Show the plot
fig.show()

# Create a combined layout with subplots. 

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import os

# Create a simplified subplot layout with 1 row and 2 columns
fig = make_subplots(
    rows=1, 
    cols=2,
    subplot_titles=(
        'Net Operating Cashflow', 
        'Cashflow Waterfall'
    ),
    specs=[
        [{"type": "bar"}, {"type": "waterfall"}]
    ],
    horizontal_spacing=0.12
)

# Get unique years from CashFlow
years = sorted(CashFlow['yearReport'].unique())

# 1. Net Operating Cashflow (Bar Chart) - Left
# For each year, calculate the average Net cash inflows/outflows from operating activities
ocf_data = CashFlow.groupby('yearReport')['Net cash inflows/outflows from operating activities'].mean() / 1000  # Convert to billions
ocf_years = ocf_data.index.tolist()
ocf_values = ocf_data.values.tolist()

# Add bar chart
fig.add_trace(
    go.Bar(
        x=ocf_years,
        y=ocf_values,
        text=[f'{v:.2f}B' for v in ocf_values],
        textposition='outside',
        marker_color='rgb(66, 135, 245)',
        marker_line_color='lightgrey',
        marker_line_width=1.5,
        opacity=0.8,
        name='Net Operating Cash Flow'
    ),
    row=1, col=1
)

# 2. Cashflow Waterfall - Right
# Get a representative ticker (or use one specified)
ticker = CashFlow['ticker'].iloc[0]
# Get the latest year
latest_year = max(years)

# Filter data for the specific ticker and latest year
waterfall_data = CashFlow[(CashFlow['ticker'] == ticker) & 
                          (CashFlow['yearReport'] == latest_year)]

if len(waterfall_data) > 0:
    data = waterfall_data.iloc[0]
    
    # Key cashflow components
    measures = [
        'Initial Cash',
        'Operating CF',
        'Investing CF',
        'Financing CF',
        'Final Cash'
    ]
    
    values = [
        data['Cash and cash equivalents'],
        data['Net cash inflows/outflows from operating activities'],
        data['Net Cash Flows from Investing Activities'],
        data['Cash flows from financial activities'],
        data['Cash and Cash Equivalents at the end of period']
    ]
    
    measure_types = ['absolute', 'relative', 'relative', 'relative', 'total']
    
    # Add waterfall chart
    fig.add_trace(
        go.Waterfall(
            name=f"Cashflow {latest_year}",
            orientation="v",
            measure=measure_types,
            x=measures,
            textposition="outside",
            text=[f"{x:,.1f}" for x in values],
            y=values,
            connector={"line": {"color": "rgb(63, 63, 63)"}},
        ),
        row=1, col=2
    )

# Update layout for better appearance
fig.update_layout(
    title={
        'text': f'Cash Flow Analysis for {ticker}',
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    template='plotly_white',
    height=500,  # Reduced height for simplified layout
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5
    )
)

# Update axes labels and settings
fig.update_xaxes(
    title_text="Year",
    tickangle=45,
    tickmode='array',
    tickvals=years,
    row=1, 
    col=1
)

fig.update_yaxes(title_text="Value (Billions VND)", row=1, col=1)

fig.update_xaxes(title_text="Cash Flow Components", tickangle=0, row=1, col=2)
fig.update_yaxes(title_text="Amount (Bn. VND)", row=1, col=2)

# Add waterfall year to subtitle
fig.update_annotations(
    text=f"Cashflow Waterfall ({latest_year})",
    selector={"index": 1}
)

# Create outputs directory if it doesn't exist
output_dir = './outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the figure as an HTML file
output_path = os.path.join(output_dir, 'index.html')
fig.write_html(output_path)

print(f"Cash Flow dashboard saved to {output_path}")

In [46]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import os

# Define the function to calculate ROCE and include ROE from Ratio dataframe
def calculate_roce_and_include_roe(BalanceSheet, IncomeStatement, Ratio):
    """
    Calculate ROCE and include ROE from Ratio dataframe with MultiIndex columns.
    """
    # Create a copy to avoid modifying the original dataframe
    BalanceSheet_copy = BalanceSheet.copy()
    
    # Step 1: Calculate Capital Employed
    BalanceSheet_copy['Capital Employed (Bn. VND)'] = (
        BalanceSheet_copy['Long-term borrowings (Bn. VND)'] + 
        BalanceSheet_copy['Short-term borrowings (Bn. VND)'] + 
        BalanceSheet_copy["OWNER'S EQUITY(Bn.VND)"]
    )
    
    # Step 2: Merge Balance Sheet and Income Statement
    merged_df = pd.merge(
        BalanceSheet_copy[['ticker', 'yearReport', 'Capital Employed (Bn. VND)']],
        IncomeStatement[['ticker', 'yearReport', 'Operating Profit/Loss']],
        on=['ticker', 'yearReport'],
        how='inner'
    )
    
    # Step 3: Calculate ROCE
    merged_df['ROCE'] = merged_df['Operating Profit/Loss'] / merged_df['Capital Employed (Bn. VND)']
    
    # Select columns for ROCE calculation
    ROCE_df = merged_df[['ticker', 'yearReport', 'Operating Profit/Loss', 
                         'Capital Employed (Bn. VND)', 'ROCE']]
    ROCE_df = ROCE_df.rename(columns={'Operating Profit/Loss': 'EBIT (Bn. VND)'})
    
    # Step 4: Create a simplified version of Ratio DataFrame for merging
    # Extract the ticker, year, and ROE columns
    ratio_simple = pd.DataFrame({
        'ticker': Ratio[('Meta', 'CP')],
        'yearReport': Ratio[('Meta', 'Năm')],
        'ROE': Ratio[('Chỉ tiêu khả năng sinh lợi', 'ROE (%)')]
    })
    
    # Step 5: Merge with simplified Ratio dataframe to include ROE
    ROCE_df = pd.merge(
        ROCE_df,
        ratio_simple[['ticker', 'yearReport', 'ROE']],
        on=['ticker', 'yearReport'],
        how='left'
    )
    
    return ROCE_df

# Create a subplot layout with 2 rows and 2 columns with increased vertical spacing
fig = make_subplots(
    rows=2, 
    cols=2,
    subplot_titles=(
        'Net Operating Cashflow', 
        'ROIC (%) vs WACC (%) with Economic Value Zone',
        'Cashflow Waterfall',
        'ROCE vs ROE Comparison'
    ),
    specs=[
        [{"type": "xy"}, {"type": "xy"}],
        [{"type": "waterfall"}, {"type": "xy", "secondary_y": True}]
    ],
    vertical_spacing=0.15,  # Increased vertical spacing
    horizontal_spacing=0.08
)

# Get unique years from CashFlow
years = sorted(CashFlow['yearReport'].unique())

# 1. Net Operating Cashflow (Bar Chart) - Top Left
# For each year, calculate the average Net cash inflows/outflows from operating activities
ocf_data = CashFlow.groupby('yearReport')['Net cash inflows/outflows from operating activities'].mean() / 1000  # Convert to billions
ocf_years = ocf_data.index.tolist()
ocf_values = ocf_data.values.tolist()

# Add bar chart
fig.add_trace(
    go.Bar(
        x=ocf_years,
        y=ocf_values,
        text=[f'{v:.2f}B' for v in ocf_values],
        textposition='outside',
        marker_color='rgb(66, 135, 245)',
        marker_line_color='lightgrey',
        marker_line_width=1.5,
        opacity=0.8,
        name='Net Operating Cash Flow'
    ),
    row=1, col=1
)

# 2. ROIC vs WACC Chart with Economic Value Zone - Top Right
# Get the years from both DataFrames
ratio_years = Ratio[('Meta', 'Năm')].astype(int).values
result_years = result_df['yearReport'].astype(int).values

# Find common years
common_years = sorted(list(set(ratio_years).intersection(set(result_years))))

# Filter and sort both DataFrames by common years
ratio_filtered = Ratio[Ratio[('Meta', 'Năm')].astype(int).isin(common_years)]
ratio_filtered = ratio_filtered.sort_values(('Meta', 'Năm'))

result_filtered = result_df[result_df['yearReport'].astype(int).isin(common_years)]
result_filtered = result_filtered.sort_values('yearReport')

# Calculate values using the filtered and aligned data
wacc_mean = result_filtered['wacc_market_based'].mean() * 100
roic_values = ratio_filtered[('Chỉ tiêu khả năng sinh lợi', 'ROIC (%)')] * 100
wacc_values = result_filtered['wacc_market_based'] * 100

# Add ROIC line
fig.add_trace(
    go.Scatter(
        x=ratio_filtered[('Meta', 'Năm')],
        y=roic_values,
        name='ROIC (%)',
        line=dict(color='blue')
    ),
    row=1, col=2
)

# Add WACC line
fig.add_trace(
    go.Scatter(
        x=result_filtered['yearReport'],
        y=wacc_values,
        name='WACC (%)',
        line=dict(color='green')
    ),
    row=1, col=2
)

# Add horizontal line at WACC mean
fig.add_shape(
    type="line",
    x0=min(common_years),
    x1=max(common_years),
    y0=wacc_mean,
    y1=wacc_mean,
    line=dict(color="red", dash="dash"),
    row=1, col=2
)

# Add WACC mean annotation
fig.add_annotation(
    x=max(common_years),
    y=wacc_mean,
    text=f"WACC Mean: {wacc_mean:.2f}%",
    showarrow=False,
    xanchor="right",
    font=dict(color="red"),
    row=1, col=2
)

# Only add shaded area if we have data points
if len(roic_values) > 0:
    # Create x values for the shaded area
    x_combined = np.concatenate([ratio_filtered[('Meta', 'Năm')], ratio_filtered[('Meta', 'Năm')][::-1]])
    y_upper = np.where(roic_values > wacc_mean, roic_values, wacc_mean)
    y_lower = np.full_like(roic_values, wacc_mean)
    y_combined = np.concatenate([y_upper, y_lower[::-1]])

    # Add the shaded area
    fig.add_trace(
        go.Scatter(
            x=x_combined,
            y=y_combined,
            fill='toself',
            fillcolor='rgba(0, 100, 255, 0.2)',
            line=dict(width=0),
            showlegend=True,
            name='Economic Value',
            legendgroup='economic_value',
            hoverinfo='skip'
        ),
        row=1, col=2
    )

# Add Economic Value annotation in upper left corner of the ROIC vs WACC plot
fig.add_annotation(
    x=0.52,  # Adjusted for subplot positioning
    y=0.95,  # Top side
    xref="paper",  # Use paper coordinates (0-1)
    yref="paper",  # Use paper coordinates (0-1)
    text="<b>Economic Value</b>",
    showarrow=False,
    font=dict(
        size=12,
        color="blue"
    ),
    bgcolor='rgba(0, 100, 255, 0.2)',  # Same as fill color
    bordercolor="blue",
    borderwidth=1,
    borderpad=4,
    opacity=0.8,
    xanchor="left",  # Left-align the text
    yanchor="top"    # Anchor at the top
)

# 3. Cashflow Waterfall - Bottom Left
# Get a representative ticker (or use one specified)
ticker = CashFlow['ticker'].iloc[0]
# Get the latest year
latest_year = max(years)

# Filter data for the specific ticker and latest year
waterfall_data = CashFlow[(CashFlow['ticker'] == ticker) & 
                          (CashFlow['yearReport'] == latest_year)]

if len(waterfall_data) > 0:
    data = waterfall_data.iloc[0]
    
    # Key cashflow components
    measures = [
        'Initial Cash',
        'Operating CF',
        'Investing CF',
        'Financing CF',
        'Final Cash'
    ]
    
    values = [
        data['Cash and cash equivalents'],
        data['Net cash inflows/outflows from operating activities'],
        data['Net Cash Flows from Investing Activities'],
        data['Cash flows from financial activities'],
        data['Cash and Cash Equivalents at the end of period']
    ]
    
    measure_types = ['absolute', 'relative', 'relative', 'relative', 'total']
    
    # Add waterfall chart
    fig.add_trace(
        go.Waterfall(
            name=f"Cashflow {latest_year}",
            orientation="v",
            measure=measure_types,
            x=measures,
            textposition="outside",
            text=[f"{x:,.1f}" for x in values],
            y=values,
            connector={"line": {"color": "rgb(63, 63, 63)"}},
        ),
        row=2, col=1
    )

# 4. ROCE vs ROE Comparison - Bottom Right
# Calculate ROCE and ROE using the provided function
ROCE_df = calculate_roce_and_include_roe(BalanceSheet, IncomeStatement, Ratio)

# Filter for the specific ticker
ticker_data = ROCE_df[ROCE_df['ticker'] == ticker].sort_values('yearReport')

# Convert ROCE to percentage
ticker_data['ROCE_PCT'] = ticker_data['ROCE'] * 100

# Convert ROE to percentage if it's not already
# Check if ROE values are already in percentage (typically >1 if in %)
if ticker_data['ROE'].max() < 1:  # If ROE is in decimal form (e.g., 0.15 for 15%)
    ticker_data['ROE_PCT'] = ticker_data['ROE'] * 100
else:
    ticker_data['ROE_PCT'] = ticker_data['ROE']  # If already in percentage form

# Add ROCE trace (using percentage values) - REMOVED text labels
fig.add_trace(
    go.Scatter(
        x=ticker_data['yearReport'],
        y=ticker_data['ROCE_PCT'],
        name='ROCE (%)',
        mode='lines+markers',  # Removed 'text' from mode
        line=dict(color='rgba(0, 117, 210, 0.9)', width=3),
        marker=dict(
            size=10, 
            color='rgba(0, 117, 210, 0.9)',
            line=dict(color='white', width=2)
        )
    ),
    row=2, col=2,
    secondary_y=False
)

# Add shaded area under ROCE for emphasis
fig.add_trace(
    go.Scatter(
        x=ticker_data['yearReport'],
        y=ticker_data['ROCE_PCT'],
        name='ROCE Area',
        mode='none',
        fill='tozeroy',
        fillcolor='rgba(0, 117, 210, 0.1)',
        showlegend=False,
        hoverinfo='skip'
    ),
    row=2, col=2,
    secondary_y=False
)

# Add ROE trace using percentage values - REMOVED text labels
fig.add_trace(
    go.Scatter(
        x=ticker_data['yearReport'],
        y=ticker_data['ROE_PCT'],
        name='ROE (%)',
        mode='lines+markers',  # Removed 'text' from mode
        line=dict(color='rgba(220, 20, 60, 0.9)', width=3),
        marker=dict(
            size=10, 
            symbol='diamond',
            color='rgba(220, 20, 60, 0.9)',
            line=dict(color='white', width=2)
        )
    ),
    row=2, col=2,
    secondary_y=True
)

# Add shaded area under ROE for emphasis
fig.add_trace(
    go.Scatter(
        x=ticker_data['yearReport'],
        y=ticker_data['ROE_PCT'],
        name='ROE Area',
        mode='none',
        fill='tozeroy',
        fillcolor='rgba(220, 20, 60, 0.1)',
        showlegend=False,
        hoverinfo='skip'
    ),
    row=2, col=2,
    secondary_y=True
)

# Financial Leverage Interpretation annotation has been removed

# Update layout for better appearance
fig.update_layout(
    title={
        'text': f'Financial Analysis Dashboard for {ticker}',
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    template='plotly_white',
    height=1000,  # Increased height to accommodate annotations and avoid overlap
    width=1200,
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.17,  # Lowered legend position to avoid overlap
        xanchor="center",
        x=0.5,
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    ),
    hovermode='x unified',
    margin=dict(t=100, b=180, l=80, r=80),  # Increased bottom margin
)

# Update axes labels and settings
fig.update_xaxes(
    title_text="Year",
    tickangle=45,
    tickmode='array',
    tickvals=years,
    row=1, 
    col=1
)

fig.update_yaxes(title_text="Value (Billions VND)", row=1, col=1)

fig.update_xaxes(
    title_text="Year",
    tickangle=45,
    tickmode='array',
    tickvals=common_years,
    row=1, 
    col=2
)

fig.update_yaxes(title_text="Percentage (%)", row=1, col=2)

fig.update_xaxes(title_text="Cash Flow Components", tickangle=0, row=2, col=1)
fig.update_yaxes(title_text="Amount (Bn. VND)", row=2, col=1)

# Update ROCE vs ROE axes
fig.update_xaxes(
    title_text="Year",
    tickangle=45,
    tickmode='array',
    tickvals=ticker_data['yearReport'],
    showgrid=True,
    gridwidth=1,
    gridcolor='rgba(211, 211, 211, 0.4)',
    showline=True,
    linewidth=2,
    linecolor='rgba(0, 0, 0, 0.3)',
    row=2, 
    col=2
)

fig.update_yaxes(
    title_text="ROCE (%)",
    title_font=dict(color='rgba(0, 117, 210, 1)'),
    showgrid=True,
    gridwidth=1, 
    gridcolor='rgba(211, 211, 211, 0.4)',
    showline=True,
    linewidth=2, 
    linecolor='rgba(0, 117, 210, 0.7)',
    tickformat='.1f',
    ticksuffix='%',
    row=2, 
    col=2,
    secondary_y=False
)

fig.update_yaxes(
    title_text="ROE (%)",
    title_font=dict(color='rgba(220, 20, 60, 1)'),
    showgrid=False,
    showline=True, 
    linewidth=2, 
    linecolor='rgba(220, 20, 60, 0.7)',
    tickformat='.1f',
    ticksuffix='%',
    row=2, 
    col=2,
    secondary_y=True
)

# Add waterfall year to subtitle
fig.update_annotations(
    text=f"Cashflow Waterfall ({latest_year})",
    selector={"index": 2}
)

# Create outputs directory if it doesn't exist
output_dir = './dist'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the figure as an HTML file
output_path = os.path.join(output_dir, 'index.html')
fig.write_html(output_path)

print(f"Financial dashboard saved to {output_path}")

Financial dashboard saved to ./dist\index.html
